In [ ]:
from enum import IntEnum

import numpy as np

import GCode
import GRBL


# CNC Laser Power Settings Test 2

## Objective:

- Make more functions for testing.
- Figure out constant vs dynamic power results.
- Test between 10 & 25

# Code:

In [3]:
cnc = GRBL.GRBL(port="/dev/cnc_3018")

print("Laser Mode: {}".format(cnc.laser_mode))


Laser Mode: 1.0


In [4]:
class Tool(IntEnum):    SPINDLE = 0    LASER = 1

In [5]:
class LaserPower(IntEnum):    CONSTANT = 0    DYNAMIC = 1

In [6]:
LaserPower.CONSTANT


<LaserPower.CONSTANT: 0>

In [7]:
def init(power=LaserPower(0), feed=200, laser=25):
    program = GCode.GCode()
    program.G21()  # Metric Units
    program.G91()  # Absolute positioning.
    program.G1(F=feed)  #
    if power == LaserPower.CONSTANT:
        program.M3(S=laser)  # Laser settings
    else:
        program.M4(S=laser)  # Laser settings
    return program


In [8]:
def end():
    program = GCode.GCode()
    program.M5()  # Laser settings.
    return program


In [8]:
def square(size=20):
    program = GCode.GCode()
    program.G1(X=size)
    program.G1(Y=size)
    program.G1(X=-size)
    program.G1(Y=-size)
    return program


## Test Setup

Position the paper & other things.

In [13]:
cnc.cmd("M5")  # Laser off


['ok', 'ok']

In [12]:
# Set minimal power setting to focus and position laser
for i in range:
    cnc.cmd("M3 S255")
cnc.cmd("G1 X0")  # Laser On


['ok', 'ok']

In [51]:
def jogx(x=10):
    program = GCode.GCode()
    program.G0(X=x)
    cnc.run(program)


def jogy(y=10):
    program = GCode.GCode()
    program.G0(Y=y)
    cnc.run(program)


def jogz(z=10):
    program = GCode.GCode()
    program.G0(Z=z)
    cnc.run(program)


In [13]:
def kill_cnc():
    # TODO:
    pass


In [14]:
workspace = 220  # mm


In [15]:
square_size = 10
spacing = 5


In [16]:
test_space = square_size + spacing


In [17]:
test_space


15

In [22]:
tests = workspace / test_space
tests


14.666666666666666

In [26]:
tests = np.floor(tests)
tests


14.0

14 tests.

In [61]:
laser_pwms = np.linspace(10, 50, 14, dtype=np.uint8)
laser_pwms


array([10, 13, 16, 19, 22, 25, 28, 31, 34, 37, 40, 43, 46, 50], dtype=uint8)

In [62]:
laser_powers = [LaserPower.CONSTANT, LaserPower.DYNAMIC]


In [63]:
jogx(-1 * test_space)


In [60]:
jogy(10)


In [59]:
def init(power=LaserPower(0), feed=200, laser=25):
    program = GCode.GCode()
    program.G21()  # Metric Units
    program.G91()  # Absolute positioning.
    program.G1(F=feed)  #
    if power == LaserPower.CONSTANT:
        program.M3(S=laser)  # Laser settings
    else:
        program.M4(S=laser)  # Laser settings
    return program


for laser_power in laser_powers:
    for laser_pwm in laser_pwms:
        print("Lasers Set To: {} {}".format(laser_power, laser_pwm))
        program = (
            init(power=laser_power, laser=laser_pwm)
            + square(size=square_size)
            + end()
        )
        print(".", end="")
        cnc.run(program)
        jogx(square_size)
        jogx(spacing)
        print("")
    jogx((-1 * test_space) * tests)
    jogy(spacing)


Lasers Set To: 0 10
.
Lasers Set To: 0 13
.
Lasers Set To: 0 16
.
Lasers Set To: 0 19
.
Lasers Set To: 0 22
.
Lasers Set To: 0 25
.
Lasers Set To: 0 28
.
Lasers Set To: 0 31
.
Lasers Set To: 0 34
.
Lasers Set To: 0 37
.
Lasers Set To: 0 40
.
Lasers Set To: 0 43
.
Lasers Set To: 0 46
.
Lasers Set To: 0 50
.
Lasers Set To: 1 10
.
Lasers Set To: 1 13
.

KeyboardInterrupt: 

# Experimental Setup.

Assembled Chinese CNC 3018.
GRBL Version

In [66]:
cnc.cmd("$$")


['ok', 'error:8']

In [67]:
cnc.cmd("$#")


['ok', 'error:8']

In [68]:
cnc.cmd("$I")


['ok', 'error:8']

In [69]:
cnc.cmd("$N")


['ok', 'error:8']

# Results

## M3
- Laser needs to 'warm up'(?) on constant power mode
- Starts to smoke @ 22 on graphite.
- Starts showing up @ 25, 40 nearly completed a cut, but laser wasn't on whole cycle. 43+ all cut.
- 40 barely marked second paper.


- ``1`` Can not be seen.
- ``10`` Can not be seen.
- ``50`` Cut through 1 piece of paper & marked one under it.
- ``100`` cut through 2 pieces and light etch on clipboard.
- ``150`` Cut through 2 pieces and dark etch on clipboard.
- ``255`` & ``1024`` look identical. Etched 'well' into clipboard.


## M4

- Very itermittent. 
- ```50``` didn't cut all the way through. Almost no marking on second paper.

# Test Conclusion.

- Need to wrap Keyboard Kill with a CNC Kill.
- Need more tests.